In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [3]:
data.replace({"sentiment" : {"positive":1, "negative":0}}, inplace = True)
data

/tmp/ipykernel_36/3169135509.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment" : {"positive":1, "negative":0}}, inplace = True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [4]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [6]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [5]:
tokenizer = Tokenizer(num_words = 500)
tokenizer.fit_on_texts(train_data['review'])

max_length = max([len(review) for review in train_data['review']])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=max_length, padding='pre')
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=max_length, padding='pre')
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [10]:
# build the model
model = Sequential()
model.add(Embedding(input_dim=500, output_dim=128))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [11]:
model.build(input_shape=(None, max_length))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 13704, 128)     │        64,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 113,473 (443.25 KB)

 Trainable params: 113,473 (443.25 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [13]:
model.fit(X_train, Y_train, epochs=1, batch_size=64, validation_split=0.2)

500/500 ━━━━━━━━━━━━━━━━━━━━ 13508s 27s/step - accuracy: 0.6883 - loss: 0.5743 - val_accuracy: 0.7621 - val_loss: 0.5221


In [15]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1573s 5s/step - accuracy: 0.7615 - loss: 0.5272
Test loss: 0.524433434009552
Test Accuracy: 0.7621999979019165


In [16]:
def predict(review):
    tokens = tokenizer.texts_to_sequences([review])
    pad_tokens = pad_sequences(tokens, maxlen = max_length, padding='pre')
    predict = model.predict(pad_tokens)
    print(predict)
    sentiment = "positive" if predict[0][0] > 0.5 else "negative"
    return sentiment

In [17]:
new_reviews = ["This movie was fantastic. I loved it.", "This movie was not that good", "This movie was ok but not that good."]

for review in new_reviews:
    sentiment = predict(review)
    print(f"The sentiment of {review} is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
[[0.7063201]]
The sentiment of This movie was fantastic. I loved it. is: positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
[[0.4757416]]
The sentiment of This movie was not that good is: negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
[[0.49574453]]
The sentiment of This movie was ok but not that good. is: negative
